# Trabajo Práctico 2: Entrenamiento y evaluación de modelos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn_pandas
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from collections import defaultdict
import warnings

warnings.filterwarnings('ignore')

## Se debe elegir y definir una métrica de performance a utilizar para evaluar los modelos. Fundamentar la elección de la métrica.



En nuestro caso utilizaremos Acuracy porque funciona muy bien para los problemas de clasificación, y ademas nos permite observar aquellos datos con mayor precisión
Esta tecnica nos va a poder mostrar el porcentaje de aciertos que tuvimos.

In [2]:
# Importamos el dataset.
dataset = pd.read_csv('Video_Games_Sales.csv',encoding='UTF-8')
dataset.sample(25)

,Unnamed: 0,Game,Platform,Gener,Publisher,North_America,Europe,Japan,Rest_of _World,Global,Good_Sale,Count_Games
7121,7121,Patapon,PSP,Misc,Sony Computer Entertainment,0.33,0.25,0.07,0.16,0.81,0,617
9558,9558,Taiko Drum Master,PS2,Misc,Namco Bandai Games,0.10,0.08,0.00,0.03,0.20,0,765
2069,2069,Derby Stallion III,SNES,Sports,ASCII Entertainment,0.00,0.00,1.15,0.00,1.15,2,19
86,86,7 Wonders: Treasures of Seven,DS,Puzzle,Avanquest,0.01,0.02,0.00,0.00,0.03,0,26
1442,1442,Chou Gekijouban Keroro Gunsou 3: Tenkuu Daibou...,DS,Action,Namco Bandai Games,0.00,0.00,0.06,0.00,0.06,0,765
10721,10721,Valhalla Knights,PSP,Role-Playing,Rising Star Games,0.15,0.00,0.07,0.01,0.24,0,81
776,776,Bella Sara,DS,Misc,Codemasters,0.04,0.03,0.00,0.01,0.08,0,75
3597,3597,Gitaroo Man,PS2,Misc,THQ,0.02,0.02,0.00,0.01,0.05,0,394
8727,8727,SingStar Anthems,PS2,Misc,Sony Computer Entertainment,0.00,0.68,0.00,0.20,0.89,0,617
9353,9353,Super Mario All-Stars,SNES,Platform,Nintendo,5.99,2.15,2.12,0.29,10.55,2,665


In [3]:
dataset.dtypes

Unnamed: 0          int64
Game               object
Platform           object
Gener              object
Publisher          object
North_America     float64
Europe            float64
Japan             float64
Rest_of _World    float64
Global            float64
Good_Sale           int64
Count_Games         int64
dtype: object

In [4]:
# Limpiamos un poco el dataset
dataset.drop(columns=['Unnamed: 0', 'Platform','North_America', 'Europe', 'Japan', 'Rest_of _World'],axis=1,inplace=True)
dataset.dtypes

Game            object
Gener           object
Publisher       object
Global         float64
Good_Sale        int64
Count_Games      int64
dtype: object

In [5]:
# Transformamos la variable Gener a One-Hot Encoding
dataset.Gener.value_counts()

Gener
Action          1949
Sports          1377
Misc            1304
Role-Playing    1227
Adventure       1060
Shooter          825
Racing           771
Simulation       726
Fighting         611
Platform         588
Strategy         580
Puzzle           498
Name: count, dtype: int64

In [6]:
dummies = pd.get_dummies(dataset['Gener'], drop_first = True)
dummies.head()

,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
0,False,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,False


In [7]:
# Añadimos las variables al DataSet
dataset = pd.concat([dataset, dummies], axis = 1)

# Eliminamos la vairable original Gener
dataset = dataset.drop(columns=['Gener'])

In [8]:
dataset.sample(25)

,Game,Publisher,Global,Good_Sale,Count_Games,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
1366,Challenge Me: Maths Workout,Oxygen Interactive,0.04,0,17,False,False,False,False,True,False,False,False,False,False,False
1058,Breath of Fire: Dragon Quarter,Capcom,0.19,0,266,False,False,False,False,False,False,True,False,False,False,False
8769,Ski-Doo Snowmobile Challenge,PQube,0.22,0,24,False,False,False,False,False,True,False,False,False,False,False
6744,New Super Mario Bros.,Nintendo,29.80,2,665,False,False,False,True,False,False,False,False,False,False,False
7650,Puzzler Mind Gym 3D,Ubisoft,0.08,0,564,False,False,False,False,True,False,False,False,False,False,False
4237,Hyakka Yakou,Unknown,0.01,0,148,False,False,False,False,False,False,False,False,False,False,False
10478,Transformer: Rise of the Dark Spark (3DS),Activision,0.07,0,401,False,False,False,False,False,False,True,False,False,False,False
8134,Rumble Roses,Konami Digital Entertainment,0.39,0,637,False,True,False,False,False,False,False,False,False,False,False
1498,Classic NES Series: Pac-Man,Nintendo,0.46,0,665,False,False,False,False,True,False,False,False,False,False,False
3554,Geometry Wars: Galaxies,Vivendi Games,0.26,0,86,False,False,False,False,False,False,False,True,False,False,False


In [9]:
dataset = dataset.rename(columns={'Role-Playing':'RolePlaying'})

In [10]:
dataset.sample(5)

,Game,Publisher,Global,Good_Sale,Count_Games,Adventure,Fighting,Misc,Platform,Puzzle,Racing,RolePlaying,Shooter,Simulation,Sports,Strategy
10036,The Next Tetris,Atari,0.81,0,277,False,False,False,False,True,False,False,False,False,False,False
2794,Endgame,Empire Interactive,0.08,0,35,False,False,False,False,False,False,False,True,False,False,False
9722,Tennis no Oji-Sama: Genius Boys Academy,Konami Digital Entertainment,0.25,0,637,False,False,False,False,False,False,False,False,False,True,False
4530,Jeanne d'Arc,Sony Computer Entertainment,0.33,0,617,False,False,False,False,False,False,True,False,False,False,False
10509,Treasure Gaust: Gaust Diver Crimson Red / Deep...,Namco Bandai Games,0.03,0,765,False,False,False,False,False,False,False,False,False,False,False


## Se debe aplicar alguna técnica de feature engineering para mejorar los datos de entrada a los modelos, y mostrar la comparativa de los resultados obtenidos en cada caso. Si no es posible o útil, fundamentar el motivo por el cual no se realizará.



In [11]:
mapper = DataFrameMapper([
    (['Count_Games'], [StandardScaler()]),
    (['Adventure'], [OneHotEncoder()]),
    (['Fighting'], [OneHotEncoder()]),
    (['Misc'], [OneHotEncoder()]),
    (['Platform'], [OneHotEncoder()]),
    (['Puzzle'], [OneHotEncoder()]),
    (['Racing'], [OneHotEncoder()]),
    (['RolePlaying'], [OneHotEncoder()]),
    (['Shooter'], [OneHotEncoder()]),
    (['Simulation'], [OneHotEncoder()]),
    (['Sports'], [OneHotEncoder()]),
    (['Strategy'], [OneHotEncoder()])
])

In [12]:
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(dataset, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train.shape, validation.shape, test.shape

((6909, 16), (2303, 16), (2304, 16))

In [13]:
# Función para evaluar modelos con la tecnica de future engineering
def evaluar_Modelo(model, set_names=('train', 'validation'), Titulo=''):
    display(Titulo)
        
    final_metrics = {'Accuracy': []}
   
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name] 
        y = set_data.Good_Sale
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        
    display(pd.DataFrame(final_metrics, index=set_names))

In [14]:
#Con feature engeeniring, sin hiperparámetros.

# kNN
Modelo_KNN = Pipeline([('mapper', mapper), ('classifier', KNeighborsClassifier())])
Modelo_KNN.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN')

'kNN'

,Accuracy
train,0.839195
validation,0.815458


In [15]:
# Decision Tree
Modelo_Decision_Tree = Pipeline([('mapper', mapper), ('classifier', DecisionTreeClassifier())])
Modelo_Decision_Tree.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree, Titulo='Decision Tree')

'Decision Tree'

,Accuracy
train,0.853524
validation,0.823708


In [16]:
# Gradient Boosting
Modelo_Gradient_Boosting = Pipeline([('mapper', mapper), ('classifier', GradientBoostingClassifier(random_state=42))])
Modelo_Gradient_Boosting.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting, Titulo='Gradient Boosting')

'Gradient Boosting'

,Accuracy
train,0.844695
validation,0.832393


Segun los datos obtenidos, vemos que en KNN y en Decision Tree ahi diferencia entre train y validation, mientras que en Gradient Boosting, la misma posee una diferencia menor.

A su ves tambien podemos observar que Decision Tree posee el valor de train mas alto con respecto a los otros 2 modelos. En el caso de validation, el que posee mejor resultado es Gradient Boosting.

## Por cada modelo, se debe entrenarlo y realizar una exploración de hiper-parámetros mediante una búsqueda en grilla. Evaluar el comportamiento de cada modelo con los hiper-parámetros que mejores resultados ofrecen. En caso de ser posible, aporte conclusiones respecto a dicha comparación.

Evaluaremos los modelos con los Hiper-parámetros: 
* Para KNN: 2, 4 y 6
* Para Decision Tree y Gradient Bosting: max_depth de 5, 10 y 15

In [17]:
# kNN 2
Modelo_KNN_2 = Pipeline([('mapper', mapper), ('classifier', KNeighborsClassifier(2))])
Modelo_KNN_2.fit(train, train.Good_Sale)
predictions = Modelo_KNN_2.predict(validation)
evaluar_Modelo(Modelo_KNN_2, Titulo='kNN-2')

# kNN 4
Modelo_KNN_4 = Pipeline([('mapper', mapper), ('classifier', KNeighborsClassifier(4))])
Modelo_KNN_4.fit(train, train.Good_Sale)
predictions = Modelo_KNN_4.predict(validation)
evaluar_Modelo(Modelo_KNN_4, Titulo='kNN-4')

# kNN 6
Modelo_KNN_6 = Pipeline([('mapper', mapper), ('classifier', KNeighborsClassifier(6))])
Modelo_KNN_6.fit(train, train.Good_Sale)
predictions = Modelo_KNN_6.predict(validation)
evaluar_Modelo(Modelo_KNN_6, Titulo='kNN-6')

'kNN-2'

,Accuracy
train,0.839195
validation,0.820669


'kNN-4'

,Accuracy
train,0.838472
validation,0.818063


'kNN-6'

,Accuracy
train,0.843682
validation,0.827182


En el modelo KNN podemos notar que con un hiper-parámetro de 2, obtenemos mejores resultados que con uno de 4, pero el que mejor resultados ofrece, tanto en train como en validation es con hiper-parámetro de 6.

In [18]:
# Decision Tree 5
Modelo_Decision_Tree_5 = Pipeline([('mapper', mapper), ('classifier', DecisionTreeClassifier(max_depth = 5))])
Modelo_Decision_Tree_5.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree_5, Titulo='Decision Tree 5')

# Decision Tree 10
Modelo_Decision_Tree_10 = Pipeline([('mapper', mapper), ('classifier', DecisionTreeClassifier(max_depth = 10))])
Modelo_Decision_Tree_10.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree_10, Titulo='Decision Tree 10')

# Decision Tree 15
Modelo_Decision_Tree_15 = Pipeline([('mapper', mapper), ('classifier', DecisionTreeClassifier(max_depth = 15))])
Modelo_Decision_Tree_15.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree_15, Titulo='Decision Tree 15')

'Decision Tree 5'

,Accuracy
train,0.844551
validation,0.831524


'Decision Tree 10'

,Accuracy
train,0.847735
validation,0.830656


'Decision Tree 15'

,Accuracy
train,0.851788
validation,0.826314


En el caso de Decision Tree, podemos notar que a medida que incrementamos el hiper-parámetro, el resultado de train incrementa, mientras que el resultado de validations disminuye.

Con esto podemos concluir que mientras mayor sea el hiper-parámetro, mayor sera la diferencia entre train y validation.

In [19]:
# Gradient Boosting 5
Modelo_Gradient_Boosting_5 = Pipeline([('mapper', mapper), ('classifier', GradientBoostingClassifier(max_depth = 5, random_state=42))])
Modelo_Gradient_Boosting_5.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting_5, Titulo='Gradient Boosting 5')

# Gradient Boosting 10
Modelo_Gradient_Boosting_10 = Pipeline([('mapper', mapper), ('classifier', GradientBoostingClassifier(max_depth = 10, random_state=42))])
Modelo_Gradient_Boosting_10.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting_10, Titulo='Gradient Boosting 10')

# Gradient Boosting 15
Modelo_Gradient_Boosting_15 = Pipeline([('mapper', mapper), ('classifier', GradientBoostingClassifier(max_depth = 15, random_state=42))])
Modelo_Gradient_Boosting_15.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting_15, Titulo='Gradient Boosting 15')

'Gradient Boosting 5'

,Accuracy
train,0.848314
validation,0.828485


'Gradient Boosting 10'

,Accuracy
train,0.853524
validation,0.823708


'Gradient Boosting 15'

,Accuracy
train,0.853524
validation,0.819800


En Gradient Boosting vemos que pasa lo mismo que en Decision Tree, solo que la variacion de los resultados es mas notable que en el caso de Decision Tree.

## Realizar experimentos que utilicen como datos de entrada representaciones intermedias de los datos (generadas por técnicas de reducción de dimensiones como PCA). Compare los resultados obtenidos contra los casos previos, interprete y proponga conclusiones.


In [20]:
# KNN-PCA

Modelo_KNN_PCA = Pipeline([('mapper', mapper), ('pca', PCA()), ('classifier', KNeighborsClassifier(6))])
Modelo_KNN_PCA.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN-PCA')

'kNN-PCA'

,Accuracy
train,0.839195
validation,0.815458


In [21]:
# Decision Tree PCA

Modelo_Decision_Tree_PCA = Pipeline([('mapper', mapper), ('pca', PCA()), ('classifier', DecisionTreeClassifier(max_depth = 5))])
Modelo_Decision_Tree_PCA.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree_PCA, Titulo='Decision Tree PCA')

'Decision Tree PCA'

,Accuracy
train,0.844261
validation,0.832393


In [22]:
# Gradient Boosting PCA

Modelo_Gradient_Boosting_PCA = Pipeline([('mapper', mapper), ('pca', PCA()), ('classifier', GradientBoostingClassifier(max_depth = 5, random_state=42))])
Modelo_Gradient_Boosting_PCA.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting_PCA, Titulo='Gradient Boosting')

'Gradient Boosting'

,Accuracy
train,0.848748
validation,0.827182


Para realizar el analisis con PCA, utilizamos los Hiper-parámetros que consideramos mejores en cada modelo de los que utilizamos.

En el caso de KNN notamos que el resultado es igual que cuando probamos el modelo sin hiper-parámetros.

Con Decision Tree nos dio el mismo resultado anteriormente visto con el modelo con el mismo hiper-parámetro.

Y por ultimo, con Gradient Boosting, obserbamos que nos dio un resultado parecido al anterior pero con una leve desviacion.

## Determinar el valor final de la métrica que podría ser informado al cliente, utilizando técnicas que permitan obtener un valor lo más realista posible. Fundamentar y considerar no solo el rendimiento del modelo en su elección, sino también cuestiones como interpretabilidad, tiempos de entrenamiento, etc.



El modelo que utilizaremos para informarle al cliente va a ser Decision Tree con una profundidad maxima de 5 y con PCA.

Este modelo fue elegido porque fue el que menos diferencia entre los valores optenidos en train y validation, como tambien por ser el que mejor resultado nos mostro con el valor de validation.

El valor final que informaremos sera de 83,23%